In [1]:
from ajpastor.dd_functions import *
from pseries_basis import *
from pseries_basis.ore import get_differential_algebra, solution, required_init
%display latex

# Changing initial values after compatibility
$\def\kk{\mathbb{K}} \def\nn{\mathbb{N}} \def\cB{\mathcal{B}} \def\cR{\mathcal{R}}$
In `pseries_basis` we work with formal power series $\kk[[x]]$ and compatible bases. A basis $\cB = \{P_k(x)\}_k$ of the formal power series is said to be compatible with a linear operator $L$ if it satisfies:
$$L P_k(x) = \sum_{i=-A}^B \alpha_{i,k} P_{k+i}(x).$$

When we have a solution $y(x)$ for the equation $L y(x) = 0$, and $L$ is compatible with $\cB$, then we can write $y(x) = \sum_k c_k P_k(x)$ and the sequence $c_k$ satisfies a D-finite equation: $$\cR_\cB(L) \cdot c_k = 0.$$

When working with a D-finite sequence or function, we provide a difference/differential operator $L$ such that $L y(x) = 0$ and some initial conditions (the values $y(0), y'(0), y''(0),\ldots$ in the differential case and $y(0), y(1), y(2), \ldots$ in the difference case). Then, it is natural to ask, given a basis $\cB$ and a D-finite sequence, *can we provide a full description of the associated sequence $(c_k)_k$?* Or, equivalently, **can we compute the initial terms of the sequence $(c_k)_k$ to define this sequence together with the operator $\cR_\cB(L)$?**

This file contains an explanation on how we can do this and the results is implemented in the method `convert_initial_terms` of the class `PolyBasis`.

## **1. Relations among cannonical basis**

The main structure for the package `pseries_basis` is the ring of formal power series $\kk[[x]]$. This is equivalent to analyze the ring of sequences $\kk^\nn$. The main basis in $\kk[[x]]$ as a $\kk$-vector space is the power basis $\{1, x, x^2, \ldots\}$. However, considering the main isomorphism between the sequences and formal power series:
$$\begin{array}{rrcl}\phi: & \kk^\nn & \longleftrightarrow & \kk[[x]]\\ & (a_n)_n & \mapsto & \sum_n \left(a_n x^n\right)\end{array}.$$
In this relation among the two rings, both cannonical bases coincide. But there is another map between these two rings (or at least part of $\kk[[x]]$) which is useful for some interpretations of solutions for lienar recurrences:
$$\begin{array}{rrcl}\psi: & \kk[[x]] & \longleftrightarrow & \kk^\nn\\ & f(x) & \mapsto & (f(n))_n\end{array}.$$
For this relation, we have that the power basis is not map to the cannonical basis anymore:
* $1 \mapsto (1,1,1,1,\ldots)$.
* $x \mapsto (1,2,3,4,\ldots)$.
* $x^2 \mapsto (1,4,9,16,\ldots)$.
* $x^3 \mapsto (1,8,27,64,\ldots)$.

We can see that the relation between both basis is now **not finite**. This means that the change of coordinate may not be possible without extra conditions.

In both cases the relations between the main basis of both rings are different and understanding this relation between the sequence and the formal power series is critical to relate with the initial values of the associated recurrence $(c_k)_k$.

## **2. The associated system**

In the main theorem of [the paper](add_link), where the isomorphism between the compatible operators $L$ with a basis $\cB$ and the recurrence operators $\mathcal{E}$ was proven, we applied the compatible formula (see above) with a power series expansion w.r.t. $\cB$ and recombine everything to botain the recurrence $\cR_\cB(L)$. To study the first terms of the new sequence (the expansion w.r.t. $\cB$), we need to study a system of equations. These equations depends highly on how the sequence is defined in $\kk[[x]]$:

### *2.1 Derivation case*

When we have a linear differential operator $L$ and consider solutions $y(x) \in \kk[[x]]$, we usually provide the following iniital conditions:
$$y(0), y'(0), y''(0), \ldots$$

This is related with the fact that we are considering the first isomorphim (see $\phi$). Let us consider an infinite matrix $M$ where the $i$-th row is the expansion of $\phi^{-1}(P_k(x))$ (i.e., their sequence associates). If $P_k(x) = \sum_{n} P_{k,n} x^n$, then:
$$M = \begin{pmatrix}P_{0,0} & P_{0,1} & P_{0,2} & \ldots\\ P_{1,0} & P_{1,1} & P_{1,2} & \ldots \\ P_{2,0} & P_{2,1} & P_{2,2} & \ldots\\ P_{3,0} & P_{3,1} & P_{3,2} & \ldots\\ \vdots & \vdots & \vdots & \ddots\end{pmatrix}.$$

Hence, if we write $\mathbf{y} = (y_0, y_1,\ldots) = \phi^{-1}(y(x))$ and $\mathbf{c} = (c_0,c_1,\ldots)$ where $y(x) = \sum_k c_k P_k(x)$, we have the following identity:
$$\mathbf{y} = \mathbf{c}M.$$

**Definitions** Let $\cB = \{P_k(x)\}_k$ be a basis of $\kk[[x]]$. We say:
 * $\cB$ is a **polynomial basis** if $P_k(x) \in \kk[x]$ with degree $k$.
 * $\cB$ is an **order basis** if the order (as a formal power series) of $P_k(x)$ is exactly $k$.
    
**Remark** If $\cB$ is a polynomial (resp. an order basis), then $M$ has a lower (resp. upper) triangular shape. Hence for any fixed $n \in \nn$, we can compute the values of $(c_0, \ldots, c_n)$ using the values $(y_0,\ldots, y_n)$ and the first $(n+1)\times (n+1)$ submatrix of $M$.

*Implementation:* the method `basis_matrix` of any `PSBasis` allows to obtain the matrix $M$ for any number of rows and columns. Hence computing the initial values for $(c_k)_k$ for *polynomial* or *order* basis is a straightforward computation. See method `convert_cannonical` of the class `PSBasis` for further information.

#### *2.1.1 Example with Power Basis*

The Power Basis $\mathcal{P} = \{x^k\}_k$ is very simple in this regard since it is the cannonical basis. Hence the matrix $M$ is an infinite identity matrix. We can see here by taking a differential equation, and comparing the sequence generated by the differential equation and the sequence generated by the associated recurrence equation:

In [2]:
P = PowerBasis(); OD, (x, D) = get_differential_algebra()
L = (x-1)*D^3 - (x+1)*D^2 + D - (x-2)^2 # differential operator
f = DFinite.element(L, [0,1,2]) # D-finite representation of the sequence --> method "sequence" allows to obtain the sequence

We now transform the differential equation to a recurrence equation for the sequence $(c_k)_k$ when we express the solution `f` as the sum 
$$f(x) = \sum_k c_k x^k.$$

In [3]:
rec_op = P.recurrence(L, cleaned=True); req = required_init(rec_op)
seq = solution(rec_op, f.sequence(req, True))

We can comapre that these two sequences are the same:

In [4]:
[seq(i) for i in range(10)] == f.sequence(10, True)

True

On the other hand, the matrix $M$ can be obtained from the basis $\mathcal{P}$:

In [5]:
P.basis_matrix(5)

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

In [6]:
P.convert_cannonical(f.sequence(req, True), req)

(0, 1, 1, -1/6, -1/6, -1/40)

#### *2.1.2 Example with Hermite basis*

Another type of basis that are compatible with the derivation are Orthogonal polynomial bases. As an example, we are going to use the same operator as before but now we are going to expand the solution in terms of Hermite polynomials:

In [7]:
H = HermiteBasis(); H

Hermite Basis (1, 2*x, 4*x^2 - 2,...)

Let us first compute the recurrence equation for the sequence $c_k$ that can be defined by writing the solution $f(x) = \sum_k c_k H_k(x)$:

In [9]:
rec_op = H.recurrence(L, cleaned=True); req = required_init(rec_op); print(req)
rec_op

7


(8*n^4 + 144*n^3 + 952*n^2 + 2736*n + 2880)*Sn^6 + (-12*n^3 - 144*n^2 - 564*n - 720)*Sn^5 + (4*n^3 + 31*n^2 + 69*n + 36)*Sn^4 + (-2*n^2 - 4*n + 6)*Sn^3 + (-n - 13/2)*Sn^2 + 2*Sn - 1/4

The matrix $M$ now changes for this type of basis, but it is still a triangular matrix:

In [23]:
H.basis_matrix(5,5)

[  1   0   0   0   0]
[  0   2   0   0   0]
[ -2   0   4   0   0]
[  0 -12   0   8   0]
[ 12   0 -48   0  16]

In [10]:
many_vals = f.sequence(50, True)

In [11]:
[H.convert_cannonical(many_vals, i) for i in range(1,10)]

[(0),
 (0, 1/2),
 (1/2, 1/2, 1/4),
 (1/2, 3/8, 1/4, -1/48),
 (3/8, 3/8, 1/8, -1/48, -1/96),
 (3/8, 21/64, 1/8, -7/192, -1/96, -1/1280),
 (7/16, 21/64, 7/32, -7/192, 1/192, -1/1280, 1/1920),
 (7/16, 103/256, 7/32, 1/1536, 1/192, 3/1024, 1/1920, 19/215040),
 (231/512, 103/256, 63/256, 1/1536, 37/3072, 3/1024, 1/1024, 19/215040, 1/122880)]